## Data Manipulation at Scale: Systems and Algorithms
### Assignment 2: SQL for Data Science

https://www.coursera.org/learn/data-manipulation/programming/nkglo/sql-for-data-science-assignment

Load the `reuters.db` SQLite database.

In [39]:
import sqlite3
reuters = sqlite3.connect('reuters.db')
cr = reuters.cursor()

**Problem 1: Inspecting the Reuters Dataset and Basic Relational Algebra**

**Problem 1, Part A:** Using Select

In [40]:
cr.execute("SELECT COUNT(*) FROM (SELECT * FROM frequency WHERE docid='10398_txt_earn')")
result = cr.fetchone()[0]
print result

138


In [41]:
with open('part_a.txt', 'w') as f:
    f.write(str(result))

**Problem 1, Part B:** Using Select, Project


In [42]:
cr.execute("SELECT COUNT(*) FROM (SELECT term FROM frequency WHERE docid='10398_txt_earn' AND count=1)")
result = cr.fetchone()[0]
print result

110


In [43]:
with open('part_b.txt', 'w') as f:
    f.write(str(result))

**Problem 1, Part C:** Using Union

In [44]:
cr.execute("SELECT COUNT(*) FROM ( \
                  SELECT term FROM frequency \
                  WHERE docid='10398_txt_earn' AND count=1 \
                  UNION SELECT term FROM frequency \
                  WHERE docid='925_txt_trade' AND count=1)")
result = cr.fetchone()[0]
print result

324


In [45]:
with open('part_c.txt', 'w') as f:
    f.write(str(result))

**Problem 1, Part D:** Count unique documents

In [46]:
cr.execute("SELECT COUNT(*) FROM ( \
                  SELECT docid FROM frequency \
                  WHERE term='legal' UNION \
                  SELECT docid FROM frequency WHERE term='law')")
result = cr.fetchone()[0]
print result

58


In [47]:
with open('part_d.txt', 'w') as f:
    f.write(str(result))

**Problem 1, Part E:** Find documents with >300 terms

In [48]:
cr.execute("SELECT COUNT(*) FROM ( \
                  SELECT sum(count) as wordcount, docid \
                  FROM frequency GROUP BY docid HAVING wordcount>300)")
result = cr.fetchone()[0]
print result

107


In [49]:
with open('part_e.txt', 'w') as f:
    f.write(str(result))

**Problem 1, Part F:** Count documents that contain two words

In [50]:
cr.execute("SELECT COUNT(*) FROM ( \
              SELECT docid FROM frequency WHERE term='transactions' \
              INTERSECT SELECT docid FROM frequency WHERE term='world')")
result = cr.fetchone()[0]
print result

3


In [51]:
with open('part_f.txt', 'w') as f:
    f.write(str(result))

**Problem 2: Matrix Multiplication in SQL**

In [52]:
matrix = sqlite3.connect('matrix.db')
cm = matrix.cursor()

Express AxB as a SQL query

In [53]:
cm.execute("DROP VIEW IF EXISTS ResultMatrix")

In [54]:
cm.execute("CREATE VIEW ResultMatrix AS \
    SELECT A.row_num, B.col_num, sum(A.value*B.value) as value \
    FROM A,B WHERE A.col_num=B.row_num \
    GROUP BY A.row_num, B.col_num")

Get the value at (2,3)

In [55]:
cm.execute("SELECT value FROM ResultMatrix WHERE row_num=2 AND col_num=3")
result = cm.fetchone()[0]
print result

2874


In [56]:
with open('part_g.txt', 'w') as f:
    f.write(str(result))

**Problem 3: Working with a Term-Document Matrix**

Each row of the frequency table is a document vector, with one column for each word. Multiplying the matrix by its own transpose gives a square *similarity matrix*, where each cell represents the similarity of two documents. The similarity here is just the dot product of the two document vectors.

The condition `A.docid > B.docid` ensures that each dot product is only computed once.

The notebook crashed without the LIMIT 100 clause, but you would remove this to actually use the query for stuff.

In [57]:
cr.execute("DROP VIEW IF EXISTS SimilarityMatrix")

In [58]:
cr.execute("CREATE VIEW SimilarityMatrix AS \
    SELECT A.docid as docid1, B.docid as docid2, sum(A.count*B.count) as similarity \
    FROM frequency as A, frequency as B \
    WHERE A.term=B.term AND docid2 > docid1 \
    GROUP BY docid1, docid2")

In [59]:
cr.execute("SELECT similarity FROM SimilarityMatrix WHERE docid1='10080_txt_crude' AND docid2='17035_txt_earn'")
result = cr.fetchone()[0]
print result

19


In [60]:
with open('part_h.txt', 'w') as f:
    f.write(str(result))

To search the dataset, add a document that represents the keyword query *'washington taxes treasury'* as document q.

In [61]:
cr.execute("DROP VIEW IF EXISTS FrequencyAndQuery")

In [62]:
cr.execute("CREATE VIEW FrequencyAndQuery AS \
    SELECT * FROM FREQUENCY \
    UNION SELECT 'q' as docid, 'washington' as term, 1 as count \
    UNION SELECT 'q' as docid, 'taxes' as term, 1 as count \
    UNION SELECT 'q' as docid, 'treasury' as term, 1 as count")

Now compute the similarity matrix again. Get the 20 most similar documents to the query document q.

In [63]:
cr.execute("SELECT similarity, docA, docB FROM \
                (SELECT A.docid as docA, B.docid as docB, sum(A.count*B.count) as similarity \
                    FROM FrequencyAndQuery as A, FrequencyAndQuery as B \
                    WHERE A.term=B.term AND A.docid < B.docid \
                    GROUP BY A.docid, B.docid) \
            WHERE docA='q' OR docB='q' \
            ORDER BY similarity DESC \
            LIMIT 1")
result = cr.fetchone()[0]
print result

6


In [64]:
with open('part_i.txt', 'w') as f:
    f.write(str(result))